In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

Os dados foram extraídos do site do Kaggle:

https://www.kaggle.com/schirmerchad/bostonhoustingmlnd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cursos_Udemy/otimizacao_hiperparametro/dados/housing.csv',
                    sep=',', encoding='iso-8859-1')

In [ ]:
df.head()

**Atributos previsores**

RM: é o número médio de cômodos entre os imóveis no bairro.

LSTAT: é a porcentagem de proprietários no bairro considerados de "classe baixa".

PTRATIO: é a razão entre estudantes e professores nas escolas de ensino fundamental e médio no bairro.

**Variável alvo**

MEDV: valor médio das casas

In [ ]:
df.shape

## **EXPLORAÇÃO E TRATAMENTO DOS DADOS**

Valores Missing (NAN)

In [ ]:
# RELAÇÃO DA QUANTIDADE
df.isnull().sum()

**Análise dos tipos de atributos.**

In [ ]:
# object: strings
# int64: inteiros
# float64: reais
# complex: complexos
df.dtypes

**Análise dos outliers**

In [ ]:
import plotly.express as px

In [ ]:
boxplot = px.box(df, y="RM")
boxplot.show()

In [ ]:
boxplot = px.box(df, y="LSTAT")
boxplot.show()

In [ ]:
boxplot = px.box(df, y="PTRATIO")
boxplot.show()

In [ ]:
boxplot = px.box(df, y="MEDV")
boxplot.show()

## **ANÁLISE DA CORRELAÇÃO LINEAR**

Pearson (distribuição normal)

Spearman (distribuição não normal)

Kendall (distribuição não normal com quantidade pequena de amostras)

Análise da Normalidade

Teste Shapiro-Wilk

Ho = distribuição normal : p > 0.05

Ha = distribuição != normal : p <= 0.05

In [ ]:
import scipy.stats as stats

In [ ]:
stats.shapiro(df.MEDV)

In [ ]:
estatistica, p = stats.shapiro(df.MEDV)
print('Estatística do teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

In [ ]:
estatistica, p = stats.shapiro(df.RM)
print('Estatística do teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

In [ ]:
estatistica, p = stats.shapiro(df.LSTAT)
print('Estatística do teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

In [ ]:
estatistica, p = stats.shapiro(df.PTRATIO)
print('Estatística do teste: {}'.format(estatistica))
print('p-valor: {}'.format(p))

Ho = não há corrrelação linear: p > 0,05

Ha = existe correlação linear: p <= 0,05

In [ ]:
# Spearman
coef,p = stats.spearmanr(df.MEDV, df.LSTAT)
print('Coeficiente de correlação: {}'.format(coef))
print('p-valor: {}'.format(p))

In [ ]:
plt.scatter(df.RM, df.MEDV)
plt.title('Correlação Linear')
plt.xlabel('Quantidade Média de Cômodos')
plt.ylabel('Valor Médio')
plt.grid(False)

In [ ]:
plt.scatter(df.LSTAT, df.MEDV)
plt.title('Correlação Linear')
plt.xlabel('Quantidade Classe Baixa (%)')
plt.ylabel('Valor Médio')
plt.grid(True)


In [ ]:
plt.scatter(df.PTRATIO, df.MEDV)
plt.title('Correlação Linear')
plt.xlabel('Estudantes/Professores')
plt.ylabel('Valor Médio')
plt.grid(False)

In [ ]:
correlacoes = df.corr(method='spearman')
correlacoes

# **PRÉ-PROCESSAMENTO**

## **Separação em Variáveis Dependentes e Independentes**

In [ ]:
df.head(2)

In [ ]:
independente = df.iloc[:, 0:3].values
independente

In [ ]:
independente.shape

In [ ]:
dependente = df.iloc[:, 3].values
dependente

**Divisão em dados de treino e teste**

In [ ]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(independente, dependente, test_size = 0.3, random_state = 0)

In [ ]:
x_treino.shape, x_teste.shape

In [ ]:
y_treino.shape, y_teste.shape

# **SEM OTIMIZAR HIPERPARÂMETROS**

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

In [ ]:
# Desempenho validação cruzada
desempenho_cross_val = cross_val_score(estimator = SVR(),
                                       X = x_treino, 
                                       y = y_treino,
                                       scoring='neg_root_mean_squared_error',
                                       cv = 6)

print('RMSE: {}'.format(round(abs(desempenho_cross_val.mean()),1)))

Opções do scoring: https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

# **PRINCIPAIS HIPERPARÂMETROS DE VETORES DE SUPORTE (SVR)**

In [ ]:
from sklearn.svm import SVR

In [ ]:
SVR?

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html 

# **GRID SEARCH**

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

In [ ]:
# Determinação do espaço de hiperparâmetros
param_grid = dict(
        kernel=['linear', 'poly', 'rbf', 'sigmoid', 'precompute'],
        epsilon=[0.001,0.01,0.1,0.5,1,5],
        C = [0.25, 0.50, 0.75, 1, 2, 10, 100]
    )

In [ ]:
# Gerando o modelo
svr = SVR()

In [ ]:
# Configurando a procura com o Grid search
grid_search = GridSearchCV(svr, param_grid, scoring='neg_root_mean_squared_error', cv=6)

In [ ]:
# Configurando os melhores hiperparâmetros
grid_search.fit(x_treino, y_treino)

In [ ]:
# Melhores Hiperparâmetros
grid_search.best_params_

In [ ]:
print('Melhor Kernel: {}'.format(grid_search.best_params_['kernel']))
print('Melhor parâmetro de regularização: {}'.format(grid_search.best_params_['C']))
print('Melhor epsilon: {}'.format(grid_search.best_params_['epsilon']))
print('RMSE do melhor modelo: {}'.format(round(abs(grid_search.best_score_),1)))

In [ ]:
# Relação dos dados para todos os modelos
results = pd.DataFrame(grid_search.cv_results_)
print(results.shape)
results.head(3)

In [ ]:
# Ordenando os melhores resultados
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_kernel', 'param_C', 'param_epsilon', 'mean_test_score', 'std_test_score']].head()

# **RANDOM SEARCH**

In [ ]:
from scipy import stats

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Gerando o modelo
svr = SVR()

In [ ]:
# Determinação do espaço de hiperparâmetros
param_random = dict(
        kernel=['linear', 'poly', 'rbf', 'sigmoid', 'precompute'],
        epsilon=stats.uniform(0.001, 100),
        C = stats.uniform(0.01, 10)
    )

In [ ]:
# Configurando a procura com o Random Search
random_search = RandomizedSearchCV(svr,
                            param_random,
                            scoring='neg_root_mean_squared_error',
                            cv=6,
                            random_state=0,
                            n_iter=200)

In [ ]:
# Analisando os hiperparâmetros
random_search.fit(x_treino, y_treino)

In [ ]:
# Melhores hiperparâmetros

random_search.best_params_

In [ ]:
# Resultados:
print('Melhor kernel: {}'.format(random_search.best_params_['kernel']))
print('Melhor parâmetro de regularização: {}'.format(random_search.best_params_['C']))
print('Melhor epsilon: {}'.format(random_search.best_params_['epsilon']))
print('RMSE do melhor modelo: {}'.format(round(abs(random_search.best_score_),1)))

In [ ]:
# Resultados para todos os modelos

results = pd.DataFrame(random_search.cv_results_)

print(results.shape)

results.head()

In [ ]:
# Ordenando os modelos com melhores desempenhos
results.sort_values(by='mean_test_score', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[['param_kernel', 'param_C', 'param_epsilon','mean_test_score', 'std_test_score']].head(10)

In [ ]:
print('RMSE do melhor modelo: {}'.format(round(abs(random_search.best_score_),1)))

# **OTIMIZAÇÃO BAYESIANA**

In [ ]:
pip install scikit-optimize

In [ ]:
# Importando o Bayes Search com cross-validation:
from skopt import BayesSearchCV

In [ ]:
from sklearn.svm import SVR

In [ ]:
# Importando espaços de busca
from skopt.space import Integer, Real, Categorical

In [ ]:
# Definindo a otimização Bayesiana:
bayes_search = BayesSearchCV(estimator = SVR(),
                              search_spaces = {'kernel':Categorical(['linear', 'poly', 'rbf', 'sigmoid']),# precompute não está relacionado
                                               'C': Real(0.01,10),
                                               'epsilon':Real(0.001,100)},
                              scoring='neg_root_mean_squared_error', 
                              cv = 6,
                              n_iter = 100)

In [ ]:
# Realizando a otimização Bayesiana
bayes_search.fit(x_treino, y_treino)

In [ ]:
# Resultados:
print('Melhor kernel: {}'.format(bayes_search.best_params_['kernel']))
print('Melhor parâmetro de regularização: {}'.format(bayes_search.best_params_['C']))
print('Melhor epsilon: {}'.format(bayes_search.best_params_['epsilon']))
print('RMSE do melhor modelo: {}'.format(round(abs(bayes_search.best_score_),1)))